In [1]:
from analysis import MatchAnalysis
from classification import Classification
from ranking import Ranking
from temp.football_predictions import FootballPredictions

In [4]:
r = Ranking("Serie A", '2021-2022')
r.read_matches(seasons=4)
ma = MatchAnalysis("files/Serie A/21_22 statistics.csv")
ma.set_ranking(r)

ma.divide_and_merge_home_away()
ma.create_team_dataset()
ma.split_and_set_avg(True, 5)

fp = FootballPredictions(ma.diff_dataset)

## Lettura e pulizia degli articoli

In [ ]:
"""fp.read_all_predictions(False)
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates()
fp.df
"""

'fp.read_all_predictions(False)\nfp.recovery_games()\nfp.matches_not_found()\nfp.fix_dates()\nfp.df\n'

## Lettura degli articoli già puliti

In [ ]:
fp.read_cleaned_predictions()

In [ ]:
from my_tokenizer import MyTokenizer

In [ ]:
mt = MyTokenizer(fp.df)
mt.feature_normalization()
mt.clean_text()

X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets()

In [ ]:
c = Classification(X_train, X_test, y_train, y_test) #utilizzando Decision Tree accuratezza di 0.802
model = c.open_model('modelli/Decision Tree tokenizer.model')

In [ ]:
model.predict(X_test, y_test)
mt.set_label_prediction(model.y_pred)

In [ ]:
ma.diff_dataset.drop(columns=['completed_passings','total_passings', 'corners'], inplace=True)

In [ ]:
for i, doc in ma.diff_dataset.iterrows():
    prediction = mt.dataset_withoutNAN[(mt.dataset_withoutNAN.home == doc.home) & (mt.dataset_withoutNAN.away == doc.away)].pred
    if len(prediction)>0:
        ma.diff_dataset.at[i, 'prediction'] = prediction.values[0]

In [ ]:
for i, doc in ma.diff_dataset.iterrows():
    prediction = mt.dataset_withoutNAN[(mt.dataset_withoutNAN.home == doc.home) & (mt.dataset_withoutNAN.away == doc.away)].pred
    if len(prediction) > 0:
        ma.diff_dataset.at[i, 'prediction'] = prediction.values[0]
    else:
        ma.diff_dataset.at[i, 'prediction'] = -1

In [ ]:
ma.diff_dataset.to_csv("dataset_completo.csv")

In [ ]:
ma.merge_()
ma.diff_dataset.head()

,home,date,result,goals,away,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards,rank_h,rank_a,prediction
0,Inter,2021-08-21,V,4.0,Genoa,7.0,4.0,0.0,0.0,18.0,11.0,-1.0,0.0,7.0,1.0,1.0
1,Torino,2021-08-21,P,-1.0,Atalanta,14.0,5.0,0.0,0.0,-6.0,3.0,0.0,0.0,2.0,3.0,2.0
2,Empoli,2021-08-21,P,-2.0,Lazio,10.0,1.0,-1.0,-1.0,-8.0,5.0,1.0,0.0,0.0,8.0,2.0
3,Hellas Verona,2021-08-21,P,-1.0,Sassuolo,-4.0,-4.0,1.0,1.0,-20.0,-5.0,1.0,1.0,1.0,6.0,0.0
4,Bologna,2021-08-22,V,1.0,Salernitana,9.0,4.0,-1.0,-1.0,44.0,-3.0,3.0,1.0,8.0,0.0,1.0


In [ ]:
c1 = Classification(ma.X_train, ma.X_test, ma.y_train, ma.y_test) 
c1.create_models()
c1.gen_report()
c1.get_higher_accuracies(0.1)